In [8]:
%load_ext autoreload
%autoreload 2
from functools import partial
import os
import keras

from datasets import example_datasets, to_numpy, tensorflow_dataset, train_val_test_split
from models import mixture_poissons,poisson_glm
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix, get_unperturbed_bpr_loss_func
from experiments import training_loop
from plotting_funcs import plot_losses, plot_frontier, plot_component_histograms, plot_winners
from bpr_trials import calc_bpr_many_trials

from distributions import ZeroInflatedDist, QuantizedNormal

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from glob import glob
import scipy

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 15000
# K for bpr
K=4
seed=12343

In [3]:
train_dataset, val_dataset, test_dataset, dist_S = example_datasets(H, T, seed=seed, train_pct=1/3.0, test_pct=1/3, return_dists=True)

In [17]:
consistent_4 = [QuantizedNormal(7, 0.1) for _ in range(4)]

highvar_4 = [ZeroInflatedDist(QuantizedNormal(10, 0.1), 1-0.55) for _ in range(4)]

powerball_4 = [ZeroInflatedDist(QuantizedNormal(100, 0.1), 0.9) for _ in range(4)]
dist_S = consistent_4 + highvar_4 +powerball_4


In [4]:

train_X_THS, train_y_TS = to_numpy(train_dataset)
val_X_THS, val_y_TS = to_numpy(val_dataset)
input_shape = (H,S)

negative_bpr_K = get_bpr_loss_func(K)
negative_bpr_K_unperturbed = get_unperturbed_bpr_loss_func(K)

In [5]:
good_model_path = '/cluster/tufts/hugheslab/kheuto01/synth_topk/save_long_plot_hist_H3/K4_thresh0.6_penalty50_lr0.05/penalized_model'
model = keras.models.load_model(good_model_path)
y_preds = model(val_X_THS)

UnimplementedError: Exception encountered when calling layer 'poisson_conv' (type Conv1D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:CPU:0}} The Conv2D op currently only supports the NHWC tensor format on the CPU. The op was given the format: NCHW [Op:Conv2D] name: 

Call arguments received by layer 'poisson_conv' (type Conv1D):
  • inputs=tf.Tensor(shape=(5000, 3, 12), dtype=float32)

In [6]:
component_preds, mixture_weights = y_preds

predict_constant, predict_variable, predict_lottery = np.zeros_like(component_preds), np.zeros_like(y_preds[0]) , np.zeros_like(y_preds[0])  
predict_constant[:,0:4,:] = 1
predict_variable[:,4:8,:] = 1
predict_lottery[:,8:,:] = 1 


bpr_constant = mix_bpr(val_y_TS, (predict_constant, mixture_weights),
                       negative_bpr_K_func=negative_bpr_K)
bpr_variable = mix_bpr(val_y_TS, (predict_variable, mixture_weights),
                          negative_bpr_K_func=negative_bpr_K)
bpr_lottery = mix_bpr(val_y_TS, (predict_lottery, mixture_weights),
                            negative_bpr_K_func=negative_bpr_K)

print(f"Perturbed- 7's: {-bpr_constant*100:.1f}, 0 or 10: {-bpr_variable*100:.1f}, Lottery {-bpr_lottery*100:.1f}")  

bpr_constant = mix_bpr(val_y_TS, (predict_constant, mixture_weights),
                       negative_bpr_K_func=negative_bpr_K_unperturbed)
bpr_variable = mix_bpr(val_y_TS, (predict_variable, mixture_weights),
                          negative_bpr_K_func=negative_bpr_K_unperturbed)
bpr_lottery = mix_bpr(val_y_TS, (predict_lottery, mixture_weights),
                            negative_bpr_K_func=negative_bpr_K_unperturbed)

print(f"Unperturbed- 7's: {-bpr_constant*100:.1f}, 0 or 10: {-bpr_variable*100:.1f}, Lottery: {-bpr_lottery*100:.1f}")  

NameError: name 'y_preds' is not defined

In [ ]:
component_preds, mixture_weights = y_preds

predict_constant, predict_variable, predict_lottery = np.zeros_like(component_preds), np.zeros_like(y_preds[0]) , np.zeros_like(y_preds[0])  
predict_constant[:,0:4,:] = 1
predict_variable[:,4:8,:] = 1
predict_lottery[:,8:,:] = 1 


bpr_constant = mix_bpr(val_y_TS, (predict_constant, mixture_weights),
                       negative_bpr_K_func=negative_bpr_K)
bpr_variable = mix_bpr(val_y_TS, (predict_variable, mixture_weights),
                          negative_bpr_K_func=negative_bpr_K)
bpr_lottery = mix_bpr(val_y_TS, (predict_lottery, mixture_weights),
                            negative_bpr_K_func=negative_bpr_K)

print(f"Perturbed- 7's: {-bpr_constant*100:.1f}, 0 or 10: {-bpr_variable*100:.1f}, Lottery {-bpr_lottery*100:.1f}")  

bpr_constant = mix_bpr(val_y_TS, (predict_constant, mixture_weights),
                       negative_bpr_K_func=negative_bpr_K_unperturbed)
bpr_variable = mix_bpr(val_y_TS, (predict_variable, mixture_weights),
                          negative_bpr_K_func=negative_bpr_K_unperturbed)
bpr_lottery = mix_bpr(val_y_TS, (predict_lottery, mixture_weights),
                            negative_bpr_K_func=negative_bpr_K_unperturbed)

print(f"Unperturbed- 7's: {-bpr_constant*100:.1f}, 0 or 10: {-bpr_variable*100:.1f}, Lottery: {-bpr_lottery*100:.1f}")  

In [18]:
mnames = ['pick_consistent','cross_ratio','pick_median','pick_variance','pick_lottery', 'pick_mean']
R = 1000000
S = 1
K=4
M = len(mnames)
scores_MR = np.zeros((M, R))
for mm, method in enumerate(mnames):
    scores_MR[mm] = calc_bpr_many_trials(
        dist_S, K=K, n_trials=R, s_samples=S, strategy=method)
    
winscore_1R = scores_MR.max(axis=0, keepdims=1)

winners_MR = np.abs(scores_MR - winscore_1R) < 0.02

print(f'K={K}')
for mm, mname in enumerate(mnames):
    print("%25s E[BPR] %0.2f won % 6d/%d trials" % (mname, scores_MR[mm].mean(), winners_MR[mm].sum(), R))

K=4
          pick_consistent E[BPR] 0.61 won  398794/1000000 trials
              cross_ratio E[BPR] 0.61 won  398794/1000000 trials
              pick_median E[BPR] 0.46 won  257585/1000000 trials
            pick_variance E[BPR] 0.46 won  257585/1000000 trials
             pick_lottery E[BPR] 0.28 won  343621/1000000 trials
                pick_mean E[BPR] 0.28 won  343621/1000000 trials


In [45]:
scores_MR[:,10]

Perturbed- 7's: 51.2, 0 or 10: 50.8, Lottery 39.8
Unperturbed- 7's: 57.8, 0 or 10: 56.6, Lottery: 27.3


In [19]:
num_tracts_per_type = 4

probability_of_10 = 0.55
probability_of_100 = 0.1

outcomes = []
probabilities = []
num_10s = []
num_100s = []
bpr_denominators = []

for hit_10s in range(num_tracts_per_type+1):
    probability_of_10s = (probability_of_10**hit_10s*(1-probability_of_10)**(num_tracts_per_type-hit_10s))*scipy.special.comb(num_tracts_per_type, hit_10s)

    for hit_100s in range(num_tracts_per_type+1):

        bpr_denominator = hit_100s*100
        if K - hit_100s > 0:
            bpr_denominator += min(hit_10s, K-hit_100s)*10
        if K -hit_100s-hit_10s>0:
            bpr_denominator += (K-hit_100s-hit_10s)*7 

        probability_of_100s = (probability_of_100**hit_100s*(1-probability_of_100)**(num_tracts_per_type-hit_100s))*scipy.special.comb(num_tracts_per_type, hit_100s)
        probabilities.append(probability_of_10s * probability_of_100s)
        bpr_denominators.append(bpr_denominator)

        num_10s.append(hit_10s)
        num_100s.append(hit_100s)



In [20]:
pick_7_bprs = []
pick_10_bprs = []
pick_100_bprs = []

for probability, denominator, hit_10s, hit_100s in zip(probabilities, bpr_denominators, num_10s, num_100s):
    pick_7_bprs.append(K*7/denominator*probability)
    pick_10_bprs.append(10*hit_10s/denominator*probability)
    pick_100_bprs.append(100*hit_100s/denominator*probability)


print(f'E[%BPR] for pick 7: {np.sum(pick_7_bprs)*100:.2f}')
print(f'E[%BPR] for pick 10: {np.sum(pick_10_bprs)*100:.2f}')
print(f'E[%BPR] for pick 100: {np.sum(pick_100_bprs)*100:.2f}')

E[%BPR] for pick 7: 60.57
E[%BPR] for pick 10: 45.87
E[%BPR] for pick 100: 27.71


In [130]:
np.sum(pick_7_bprs)

In [133]:
from math import comb

# Define the probabilities for each event
probabilities = [0.7, 0.7, 0.7, 0.7, 0.1, 0.1, 0.1, 0.1]

# Initialize the probability of all possible outcomes
total_probability = 0

# Iterate over the possible counts of successful events
for successful_events in range(5):  # From 0 to 4 successful events
    # Calculate the probability of this specific outcome
    probability_of_outcome = 1
    for prob_idx, prob in enumerate(probabilities):
        if prob_idx < 4:
            probability_of_outcome *= prob if prob_idx < successful_events else (1 - prob)
        else:
            probability_of_outcome *= prob if (4 - successful_events) <= prob_idx else (1 - prob)
    # Multiply by the number of combinations of successful events
    probability_of_outcome *= comb(4, successful_events) * comb(4, 4 - successful_events)
    # Add to the total probability
    total_probability += probability_of_outcome

print("Total probability of all possible outcomes:", total_probability)

E[%BPR] for pick 7: 57.77
E[%BPR] for pick 10: 56.43
E[%BPR] for pick 100: 27.45


In [129]:
np.sum(pick_7_bprs)

0.5777087879633688

In [105]:
scipy.special.comb(4, 1)

4.0

In [107]:
from itertools import product
product([0, 1], repeat=8)

In [117]:
from math import comb

# Define the probabilities for each event
probabilities = [0.7, 0.7, 0.7, 0.7, 0.1, 0.1, 0.1, 0.1]

# Initialize the probability of all possible outcomes
total_probability = 0

# Iterate over the possible counts of successful events
for successful_events in range(5):  # From 0 to 4 successful events
    # Calculate the probability of this specific outcome
    probability_of_outcome = 1
    for prob_idx, prob in enumerate(probabilities):
        if prob_idx < 4:
            probability_of_outcome *= prob if prob_idx < successful_events else (1 - prob)
        else:
            probability_of_outcome *= prob if (4 - successful_events) <= prob_idx else (1 - prob)
    # Multiply by the number of combinations of successful events
    probability_of_outcome *= comb(4, successful_events) * comb(4, 4 - successful_events)
    # Add to the total probability
    total_probability += probability_of_outcome

print("Total probability of all possible outcomes:", total_probability)

Total probability of all possible outcomes: 0.0003784600000000001
